# 竞争性关键词推荐

## 数据预处理
1. 从文档中读入搜索内容 --
    从比赛数据的训练集中提取出搜索记录，并以utf-8格式保存，每条记录占一行

In [15]:
#从源数据中提取出搜索词条，并保存（格式：每行一个搜索词条）
data = open("./user_tag_query.10W.TRAIN",'r',encoding='gb18030')
output_file = open("./raw_words.train",'w',encoding="utf-8")
for line in data:
    line_list = line.split('\t')
    line_list = line_list[4:]
    output_line = "\n".join(line_list)
    output_file.write(output_line + '\n')
data.close()
output_file.close()

ps：搜狗的数据集内容有点黄呀～～

2. 分词
    分词使用jieba分词工具

In [1]:
import jieba
#加载数据
train_data = open("./raw_words.train","r")
#保存结果文件
result_file = open("./cutted_words.train",'w')

for sentence in train_data:
    cut_word = jieba.cut(sentence)
    line_string = "\t".join(cut_word) + '\n'
    result_file.write(line_string)
train_data.close()
result_file.close()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.921 seconds.
Prefix dict has been built succesfully.


3. 数据清洗（去除停用词）

In [2]:
#加载数据
train_data = open("./cutted_words.train","r")
#保存结果文件
result_file = open("./linkfree_words.train",'w')

#去除链接类词条
for line in train_data:
    word_list = line.split('\t')
    if word_list[0] in ['http','https','ftp','http:']:
        continue
    line_string = "\t".join(word_list) + '\n'
    result_file.write(line_string)
train_data.close()
result_file.close()

在去除停用词之前，首先获取词频分布特征

In [1]:
#读入数据，以一维列表形式
def read_word_list(filename):
    wordlist = [];
    data_file = open(filename,'r')
    for line in data_file:
        words = line.split('\t')
        wordlist.extend(words)
    data_file.close()
    return wordlist

In [2]:
#统计词频并展示(需要空闲5G左右内存)
from collections import Counter
word_list = read_word_list('./linkfree_words.train')
count_result = Counter(word_list)
for key, val in count_result.most_common(10):
    print(key, val)


 57477034
的 1830736
什么 665848
是 568017
怎么 506838
  487404
吗 424653
图片 291070
有 280052
了 230780


In [8]:
#持久化统计结果
word_list = None
result_file = open('./count_record_v1.txt','w')
for key ,val in count_result.items():
    result_file.write(key + "\t" + str(val) + "\n")
result_file.close()
    

In [10]:
#停用词过滤器

#"的", "了", "在", "是", "我", "有", "和", "就",
#"不", "人", "都", "一", "一个", "上", "也", "很", "到", "说", "要", "去", "你",
#"会", "着", "没有", "看", "好", "自己", "这"
def stop_words_fliter(word_list):
    stop_word_list = ['的','呢','怎样','如何','什么','是'，'',' ','怎么','吗','了','有']

宠辱小说-陶喆的歌-类似宝宝穿越记恋童文-禁爱诱惑-库克抛售苹果股份-老人要1元赡养费-梦里林心如-年华小说-罪爱小说全文阅读网-玉骨销魂txt书包网-与情荡涟漪有关的小说-媚媚的幸福书包网-情歌王 古巨基-暴皇的养女-微微一笑很倾城电视剧-拉直头发多少钱-深春弄潮-荆棘花园-重生之潇洒如风-猜不透 丁当-穿越之我是还珠格格-异恋全文阅读-宝贝真乖-destiny-梦里一首歌-黛爱玲g-宝宝穿越记类似的恋童文-寄清风-类似情荡涟漪的h文-小白兔进化史-一个山一个荣念什么-娇宠宝贝书包网-赵丽颖碧瑶坐-遒怎么读-哥哥爸爸真伟大-穿越小说师徒恋肉肉文-拉直加软化-拉直需要多长时间-年华庄飞扬书包网-高h文 类似宝宝穿越记-卡门歌曲-情荡涟漪-亲亲宝贝-师徒小说养成宠文有肉-一个舟一个可念什么-拉直加软化几天可以洗-蚀骨沉沦-母女低价强买排骨-还给你一些孤单-发质不错的适合软化吗-爹爹们太腹黑-异恋-琦念-玉骨销魂 清穿-一万个悲伤徐歌阳-肉肉的古代师徒恋小说-爱的旅程小说-囚妃传全文阅读瑶池-剧组拍戏发生冲突-养女成妃系列小说-爹地的宝贝-绘蓝颜-男友总是想听我声音-摄政王养女成妃的小说-逃脱书包网-恶魔哥哥请你轻点-哥哥爸爸真伟大书包网-我的监护人老公-师徒恋小说要有肉-软化头发多少钱-沁园春长沙-一个田一个奇念什么-沧海一声笑-一个山一个争念什么-媚媚的幸福-类似小白兔进化史-暴君的粉嫩娘亲-爱情公寓歌曲有哪些-gentleman 戴爱玲-布鲁塞尔爆炸袭击-黛色霜青-陕西现冥界之花-鬼帝狂妻:纨绔大小姐-首长的宝贝-destiny歌曲-一笑倾城-命运家家-毒吻狼王爹地书包网-药仙-落瑛纷飞的小说-肉文女主养成记-哥哥请你温柔一点南宫-年轮小说全文阅读-恶女擒夫邪帝请轻轻-宝宝穿越记-gentleman什么意思-夏洛特烦恼插曲-爹地请你温柔一点全文阅读-贾静雯晒咘咘浴照-逍遥小说-养父与养女的古代言情小说-冷影凝香-绾青丝全文阅读-写游泳的作文-畸念 书包网-狂魔宠女-养女成妃类似小说

